In [1]:
import torch
import random
import numpy as np
import math
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NN_Circle_Square(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.stack = nn.Sequential(
        nn.Linear(36*36, 100),
        nn.ReLU(),
        nn.Linear(100,100),
        nn.ReLU(),
        nn.Linear(100, 10),
        nn.ReLU(),
        nn.Linear(10, 2),
    )

  def forward(self, x):
    x = self.flatten(x)
    return self.stack(x)

In [121]:
class CNN_Circle_Square(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(1, 10, 5, padding=2),
            # in_channels=1,              
            # out_channels=1,            
            # kernel_size=5,              
            # stride=1,                   
            # padding=2,
        nn.ReLU(),
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(10, 10, 3, padding=1),
        nn.ReLU(),
    )
    self.out = nn.Linear(10 * 9 * 9, 2)

  def forward(self, x):
    x = F.max_pool2d(self.conv1(x), 2)
    x = F.max_pool2d(self.conv2(x), 2)
    x = x.view(x.size(0), -1)
    return self.out(x)

In [137]:
def rand_circle(d):
  img = torch.zeros(d, d)
  r = torch.randint(4, d // 2, (1, 1))[0, 0].item() # 4 to exclude 3x3 circle which are squares
  X = torch.randint(d, (1, 2))
  x = X[0, 0].item()
  y = X[0, 1].item()
  for j in range(img.shape[1]):
    for i in range(img.shape[0]):
      if (x - j)**2 + (y - i)**2 < r**2:
        img[i, j] = 1

  return img

def rand_square(d):
  img = torch.zeros(d, d)
  a = torch.randint(4, (1, 1))[0, 0].item()
  X = torch.randint(d, (1, 2)) #left corner
  x = X[0, 0].item()
  y = X[0, 1].item()

  for j in range(img.shape[1]):
    for i in range(img.shape[0]):
      if 0 <= j - x <= a  and 0 <= i - y <= a:
        img[i, j] = 1

  return img




In [34]:
d = 36
N = 20000 #trainings sample
Xtr = torch.zeros(N, 1, d, d).to(device)
Ytr = torch.zeros(N)

for i in range(N):
  if random.randint(0, 1) == 0:
    Xtr[i, 0, :, :] = rand_circle(d)
    Ytr[i] = 0 
  else:
    Xtr[i, 0, :, :] = rand_square(d)
    Ytr[i] = 1

Ytr = torch.tensor(Ytr, dtype=int).to(device)
  

<ipython-input-34-3efd07dbdeb9>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Ytr = torch.tensor(Ytr, dtype=int).to(device)


In [66]:
model = NN_Circle_Square().to(device)

In [70]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [68]:
max_steps = 10000
batch_size = 128
lossi = []

In [75]:
for i in range(max_steps):
  ixs = torch.randint(0, Xtr.shape[0], (batch_size,))
  logits = model(Xtr[ixs])
  loss = F.cross_entropy(logits, Ytr[ixs])
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  
  if i % 1000 == 0:
    print(f"{i}/{max_steps}: {loss.item()}")

  lossi.append(loss.item())

0/10000: 0.012026275508105755
1000/10000: 0.019840827211737633
2000/10000: 0.011489000171422958
3000/10000: 0.0012812070781365037
4000/10000: 0.015139899216592312
5000/10000: 0.02446376532316208
6000/10000: 0.0014124272856861353
7000/10000: 0.022506920620799065
8000/10000: 0.010860000737011433
9000/10000: 0.01938956417143345


In [141]:
Nval = 1000 #trainings sample
Xval = torch.zeros(Nval, 1, d, d).to(device)
Yval = torch.zeros(Nval)

for i in range(Nval):
  if i < N/2:
    Xval[i, 0, :, :] = rand_circle(d)
    Yval[i] = 0 
  else:
    Xval[i, 0, :, :] = rand_square(d)
    Yval[i] = 1

Yval = torch.tensor(Yval, dtype=int).to(device)

<ipython-input-141-2048d3d7d70f>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Yval = torch.tensor(Yval, dtype=int).to(device)


In [144]:
Y = model(Xval)
probs = F.softmax(Y, dim=1)
ix = torch.multinomial(probs, num_samples=1).to(device)
ix = torch.tensor([i.item() for i in ix]).to(device)

In [145]:
sum(ix == Yval).item()/Nval

0.953

In [122]:
model_cnn = CNN_Circle_Square().to(device)

In [123]:
optimizer_cnn = torch.optim.Adam(model_cnn.parameters(), lr=1e-3)

In [124]:
max_steps = 10000
batch_size = 64
lossi = []

In [125]:
for i in range(max_steps):
  ixs = torch.randint(0, Xtr.shape[0], (batch_size,))
  logits = model_cnn(Xtr[ixs])
  loss = F.cross_entropy(logits, Ytr[ixs])
  optimizer_cnn.zero_grad()
  loss.backward()
  optimizer_cnn.step()
  
  if i % 1000 == 0:
    print(f"{i}/{max_steps}: {loss.item()}")

  lossi.append(loss.item())

0/10000: 0.6911753416061401
1000/10000: 0.14475424587726593
2000/10000: 0.05706130713224411
3000/10000: 0.06376314163208008
4000/10000: 0.05851360782980919
5000/10000: 0.008182985708117485
6000/10000: 0.007679085247218609
7000/10000: 0.014429545029997826
8000/10000: 0.0068276491947472095
9000/10000: 0.003994506783783436


In [142]:
Y = model_cnn(Xval)
probs = F.softmax(Y, dim=1)
ix = torch.multinomial(probs, num_samples=1).to(device)
ix = torch.tensor([i.item() for i in ix]).to(device)

In [143]:
sum(ix == Yval).item()/Nval

1.0